In [1]:
%matplotlib inline
import pandas as pd

In [2]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
import os
tfile = os.path.join("C:\\trader\journal\_08_August\Week_3\_0817_Friday", "Trades.csv")
tfile="TradesWithHolds.csv"
# tfile="TradesExcelEdited.csv"

#The copied path from file explorer fails
t=os.path.join("C:\trader\journal\_08_August\Week_3\_0817_Friday", "Trades.csv")


t

'C:\trader\\journal\\_08_August\\Week_3\\_0817_Friday\\Trades.csv'

In [4]:
trades = pd.read_csv(tfile)
trades

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,09:44:54,IZEA,B,1.63,6016,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
1,09:47:48,IZEA,S,1.65,3923,SMRT,NaN,TRIB5621,Margin,AUTO,115.06,NaN
2,09:45:21,IZEA,B,1.59,1830,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
3,09:35:33,MU,SS,45.77,1000,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
4,09:41:12,MU,S,46.32,750,SMRT,NaN,TRIB5621,Margin,AUTO,15.00,NaN
5,09:38:46,MU,S,46.44,250,SMRT,NaN,TRIB5621,Margin,AUTO,35.00,NaN
6,09:37:48,MU,B,46.30,1000,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
7,09:36:02,MU,B,46.00,1000,SMRT,NaN,TRIB5621,Margin,AUTO,-230.00,NaN
8,09:53:58,MU,SS,46.46,479,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
9,09:59:45,MU,B,46.88,479,SMRT,NaN,TRIB5621,Margin,AUTO,-201.18,NaN


In [5]:
def addRowsOld(dframe, numRow) :
    ll=list()
    r=list()
    for i in range(len(dframe.columns)) :
        r.append('')
        
    for i in range(numRow) :
        ll.append(r)
    newdf= pd.DataFrame(ll, columns=dframe.columns)
    
    try :
        assert(len(ll[0]) == len(dframe.columns))
    except AssertionError as ex :
        print("Coding error. Assertion is untrue:", ex)
        newdf="Data Frame is not initialized"
        return newdf
    
    dframe = dframe.append(newdf, ignore_index=True)
    
    return dframe

    

In [6]:
def createDf(dframe, numRow) :
    ''' Creates a new DataFrame with  the length numRow. Each cell is filled with empty string '''

    ll=list()
    r=list()
    for i in range(len(dframe.columns)) :
        r.append('')
        
    for i in range(numRow) :
        ll.append(r)
    newdf= pd.DataFrame(ll, columns=dframe.columns)

    return newdf

In [7]:
def addRows(dframe, numRow) :
    newdf= createDf(dframe, numRow)
        
    
    dframe = dframe.append(newdf, ignore_index=True)
    
    return dframe

    

### Make sure the hour string is 0 padded.  Should probably change these to data types. 

### Todo- This, and the duration, will get more complex with the IB report format. The html report uses a long timedate format

In [8]:
def zeroPadTimeStr(dframe, timeHeading) :
    '''Guarantee that the time format xx:xx:xx'''
    

    for i, row in dframe.iterrows():
        tm = row[timeHeading]
        tms=tm.split(":")
        if int(len(tms[0]) < 2) :
            if tms[0].startswith("0") == False :
                tm= "0" + tm
                dframe.at[i, timeHeading] = tm
    return dframe
                

### Todo. Doctor an input csv file to include fractional numer of shares for testing. Make it more modular by checking for 'HOLD'. It might be useful in a windowed version with menus to do things seperately.

In [9]:
def mkShortsNegative(dframe, side, qty) :
    ''' Fix the shares sold to be negative values. Currently assuming DAS values 'B', 'S', 'SS' for 'Side'. 
    '''
    for i, row in dframe.iterrows():
        if row[side] != 'B' and row[qty] > 0:
            dframe.at[i, qty] = ((dframe.at[i, qty]) * -1)
    return dframe

### def getListTickerDF(df) will take a dataframe that includes tickers in column 'Symb' and returns a python list of DataFrames , 1 for each ticker.

In [10]:
def getListTickerDF(dframe, tickCol) :
    ldf_tick = list()
    for ticker in dframe[tickCol].unique() :
        ldf = dframe[dframe[tickCol] == ticker]
        ldf_tick.append(ldf)
    return ldf_tick

# Notebook visual tester code

In [11]:
# ddf = createDf(trades, 0)
# ddf

In [12]:
# x = zeroPadTimeStr(trades, 'Time')
# x.head()

In [13]:
# x = mkShortsNegative(trades, 'Side', 'Qty')
# x

In [14]:
# x = getListTickerDF(trades, 'Symb')
# x[2]

# End visual tester code

In [15]:
trades = zeroPadTimeStr(trades, 'Time')

In [16]:
trades.head()       
   

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,09:44:54,IZEA,B,1.63,6016,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
1,09:47:48,IZEA,S,1.65,3923,SMRT,NaN,TRIB5621,Margin,AUTO,115.06,NaN
2,09:45:21,IZEA,B,1.59,1830,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
3,09:35:33,MU,SS,45.77,1000,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
4,09:41:12,MU,S,46.32,750,SMRT,NaN,TRIB5621,Margin,AUTO,15.00,NaN


In [17]:
trades = trades.sort_values(['Symb', 'Time'])

In [18]:
# trades


In [19]:
trades = mkShortsNegative(trades, 'Side', 'Qty')

In [20]:
trades.head()

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,09:44:54,IZEA,B,1.63,6016,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
2,09:45:21,IZEA,B,1.59,1830,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
1,09:47:48,IZEA,S,1.65,-3923,SMRT,NaN,TRIB5621,Margin,AUTO,115.06,NaN
3,09:35:33,MU,SS,45.77,-1000,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
7,09:36:02,MU,B,46.00,1000,SMRT,NaN,TRIB5621,Margin,AUTO,-230.00,NaN


### Check to see if any trades are either open now or missing the opening transaction from a preveious time. If there are, Place (ticker, numshares) tuple in a list swingTrades.

In [21]:
# def getListTickerDF
#Get the tickers used today
# ldf_tick = list()
# for ticker in trades.Symb.unique() :
#     ldf = trades[trades.Symb == ticker]
#     ldf_tick.append(ldf)
# ldf_tick[0]
ldf_tick = getListTickerDF(trades, 'Symb')
ldf_tick[0]

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,09:44:54,IZEA,B,1.63,6016,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
2,09:45:21,IZEA,B,1.59,1830,SMRT,NaN,TRIB5621,Margin,AUTO,0.00,NaN
1,09:47:48,IZEA,S,1.65,-3923,SMRT,NaN,TRIB5621,Margin,AUTO,115.06,NaN


### getTickerBalance(pyl_tick) will take a python list of DataFrames created by each ticker in Symb and will return a list of tuples of tickers that have a balance of shares wrt the day's trades.

In [22]:
# def getTickerBalance  

swingTrade = list()
for ticker in ldf_tick :
    print(ticker['Symb'].unique())
    if ticker.Qty.sum() != 0 :
        swingTrade.append((ticker['Symb'].unique()[0], ticker.Qty.sum()))

swingTrade

['IZEA']
['MU']
['ROKU']
['WMT']


[('IZEA', 3923), ('MU', -750)]

In [23]:
# swingTrade = list()
# assert(len(swingTrade) == 0)
# for ticker in ldf_tick :
#     total = 0
#     prevTicker=''
#     for i, row in ticker.iterrows() :
#         total = total + row['Qty']
#         prevTicker = row['Symb']
#     if total != 0 :
#         swingTrade.append((prevTicker, total))
#     else :
#         print("skipped {0}".format(prevTicker))




In [24]:
len(swingTrade)

2

In [25]:
swingTrade

[('IZEA', 3923), ('MU', -750)]

### For each ticker that has a balance o f shares, ask the user for informatiion. If the reslting numbers of shares is accounted for great if not, add a row in the ticker df for prior transactions  and one for shares still holding.
### This will be a dialogue where the user adjusts share numbers until we geto to the correct balance.

In [26]:

for tr in swingTrade :
    print ("There is an unbalanced amount of shares of {0} in the amount of {1}".format(tr[0], tr[1]))
    stillHolding=0
    priorTransaction=0
    for i in range(10000) :
        i=2
    while True :
        try :
            question = "How many shares of {0} are you holding now?".format(tr[0])
            stillHolding = int(input(question))
        except :
            print("please enter a number")
            continue
        break
    print ("got ",stillHolding)
    if stillHolding != tr[1] :
    
        while True :
            try :
                question = "How many shares of {0} were you holding before?".format(tr[0])
                priorTransaction = int(input(question))
            except :
                print("please enter a number")
                continue
            break
    unaccounted = priorTransaction + stillHolding - int(tr[1])
    print(stillHolding, priorTransaction, unaccounted)
    if unaccounted == 0 :
        print("That works.")
    else :
        print("There are {1} unaccounted for shares in {0}".format(tr[0], unaccounted))
    
        

There is an unbalanced amount of shares of IZEA in the amount of 3923
How many shares of IZEA are you holding now?3923
got  3923
3923 0 0
That works.
There is an unbalanced amount of shares of MU in the amount of -750
How many shares of MU are you holding now?-750
got  -750
-750 0 0
That works.



### Iterate through a Python list of dataframes selected by Symb. Locate those that are also in swingTrade--list of tuples (Symb, AmtHolding). For each Symbol in swingTrade, add a row to include the amount of shares that are currelntly held. 
### Note that this does not yet include those shares that are held before the days trading began. Redo this to remake the list of data frames from the Symbols of Swing List then make a list of data frams that  excludes those then merge them together

In [27]:
listdf = createDf(trades, 0)


In [28]:
for ldf in ldf_tick :
    found=False
    for bal in swingTrade:
        print(type(bal[0][0]))
#         print(bal[0], ldf.Symb().unique())
#         if bal[0] == ldf.Symb.unique()[0] :

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [29]:
# listdf = pd.DataFrame()
for ldf in ldf_tick :
    found=False
    for bal in swingTrade:
        if bal[0] == ldf.Symb.unique()[0] :
#             print ("Got {0} with the balance {1}". format (bal[0], bal[1]))
            found = True
            ldf = addRows(ldf, 1)
            
        
            
            for i, row in ldf.iterrows():
                
                if i == len(ldf) -1 :
                    ldf.at[i, 'Time'] = '23:59:59'
                    ldf.at[i, 'Symb'] = bal[0]
                    if bal[1] > 0 :
                        ldf.at[i, 'Side'] = "HOLD+"
                    else :
                        ldf.at[i, 'Side'] = "HOLD-"
                    ldf.at[i, 'Price'] = 0
                    ldf.at[i, 'Qty'] = bal[1]
                    ldf.at[i, 'Account'] = 'ZeroSubstance'
                    ldf.at[i, 'P / L'] = 0
    listdf = listdf.append(ldf, ignore_index = True)
        

listdf                 
                    
                    


,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,09:44:54,IZEA,B,1.63,6016,SMRT,NaN,TRIB5621,Margin,AUTO,0,NaN
1,09:45:21,IZEA,B,1.59,1830,SMRT,NaN,TRIB5621,Margin,AUTO,0,NaN
2,09:47:48,IZEA,S,1.65,-3923,SMRT,NaN,TRIB5621,Margin,AUTO,115.06,NaN
3,23:59:59,IZEA,HOLD+,0,3923,,,ZeroSubstance,,,0,
4,09:35:33,MU,SS,45.77,-1000,SMRT,NaN,TRIB5621,Short,AUTO,0,NaN
5,09:36:02,MU,B,46,1000,SMRT,NaN,TRIB5621,Margin,AUTO,-230,NaN
6,09:37:48,MU,B,46.3,1000,SMRT,NaN,TRIB5621,Margin,AUTO,0,NaN
7,09:38:46,MU,S,46.44,-250,SMRT,NaN,TRIB5621,Margin,AUTO,35,NaN
8,09:41:12,MU,S,46.32,-750,SMRT,NaN,TRIB5621,Margin,AUTO,15,NaN
9,09:53:58,MU,SS,46.46,-479,SMRT,NaN,TRIB5621,Short,AUTO,0,NaN


In [30]:
lbls = ['Tindex', 'Start', 'Time', 'Symb', 'Side', 'Price', 'Qty','Balance', 'Account', "P / L", 'Sum', 'Duration', 'Name']

In [31]:
for l in lbls :
    if l not in listdf.columns :
        listdf[l] = ''
# trades.columns
newTrades = listdf[lbls]
newTrades.copy()
newTrades.columns

Index(['Tindex', 'Start', 'Time', 'Symb', 'Side', 'Price', 'Qty', 'Balance',
       'Account', 'P / L', 'Sum', 'Duration', 'Name'],
      dtype='object')

In [32]:
newTrades

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,09:44:54,IZEA,B,1.63,6016,,TRIB5621,0,,,
1,,,09:45:21,IZEA,B,1.59,1830,,TRIB5621,0,,,
2,,,09:47:48,IZEA,S,1.65,-3923,,TRIB5621,115.06,,,
3,,,23:59:59,IZEA,HOLD+,0,3923,,ZeroSubstance,0,,,
4,,,09:35:33,MU,SS,45.77,-1000,,TRIB5621,0,,,
5,,,09:36:02,MU,B,46,1000,,TRIB5621,-230,,,
6,,,09:37:48,MU,B,46.3,1000,,TRIB5621,0,,,
7,,,09:38:46,MU,S,46.44,-250,,TRIB5621,35,,,
8,,,09:41:12,MU,S,46.32,-750,,TRIB5621,15,,,
9,,,09:53:58,MU,SS,46.46,-479,,TRIB5621,0,,,


In [33]:
nt = newTrades.sort_values(['Symb', 'Time'])
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,09:44:54,IZEA,B,1.63,6016,,TRIB5621,0,,,
1,,,09:45:21,IZEA,B,1.59,1830,,TRIB5621,0,,,
2,,,09:47:48,IZEA,S,1.65,-3923,,TRIB5621,115.06,,,
3,,,23:59:59,IZEA,HOLD+,0,3923,,ZeroSubstance,0,,,
4,,,09:35:33,MU,SS,45.77,-1000,,TRIB5621,0,,,
5,,,09:36:02,MU,B,46,1000,,TRIB5621,-230,,,
6,,,09:37:48,MU,B,46.3,1000,,TRIB5621,0,,,
7,,,09:38:46,MU,S,46.44,-250,,TRIB5621,35,,,
8,,,09:41:12,MU,S,46.32,-750,,TRIB5621,15,,,
9,,,09:53:58,MU,SS,46.46,-479,,TRIB5621,0,,,


In [34]:
prevBal = 0
for i, row in nt.iterrows():
    qty = (nt.at[i,'Qty'])
    if row['Side'].startswith("HOLD") :
        print("got it at ", qty)
        qty = qty * -1
    newBalance = qty + prevBal
    
    nt.at[i,'Balance'] = newBalance 
    prevBal = newBalance                
    
# nt

got it at  3923
got it at  -750


In [35]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
1,,,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
2,,,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
3,,,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,,,
4,,,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
5,,,09:36:02,MU,B,46,1000,0,TRIB5621,-230,,,
6,,,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
7,,,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
8,,,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,,,
9,,,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


In [36]:
#def addStartTime
newTrade = True
for i, row in nt.iterrows():
    if newTrade :
        nt.at[i,'Start'] = row['Time']
        newTrade = False
        oldTime = row['Time']
    else :
        nt.at[i, 'Start'] = oldTime
    if row['Balance'] == 0 :
        newTrade = True

nt = nt.sort_values(['Start', 'Time'])
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
4,,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
5,,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,,,
6,,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
7,,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
8,,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,,,
0,,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
1,,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
2,,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
3,,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,,,
9,,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


## Now that we have Trade index and start times everything else in this file should use filters nt[nt.Tindex == "Trade 1"] . This way each trade can be treated as an object that includes a variable number of transactions.

In [37]:
#def addTradeindex
TCount = 1
prevEndTrade = -1

for i, row in nt.iterrows():
    if len(row['Symb']) < 1 :
        break
    tradeIndex = "Trade " + str(TCount)
    if prevEndTrade == 0:
        TCount = TCount + 1
        prevEndTrade = -1
    tradeIndex = "Trade " + str(TCount)
    nt.at[i,'Tindex'] = tradeIndex 
    if row['Balance'] == 0 :
        prevEndTrade = 0
numTrades = TCount
print(numTrades)        
nt

8


,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
4,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
5,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,,,
6,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
7,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
8,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,,,
0,Trade 3,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
1,Trade 3,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
2,Trade 3,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
3,Trade 3,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,,,
9,Trade 4,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


In [38]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
4,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
5,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,,,
6,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
7,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
8,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,,,
0,Trade 3,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
1,Trade 3,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
2,Trade 3,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
3,Trade 3,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,,,
9,Trade 4,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


In [39]:
#Use filter instead of iteration
# def addTradePL
tradeTotal = 0.0
for i, row in nt.iterrows():
    if row['Balance'] != 0 :
        tradeTotal = tradeTotal + row['P / L']
    else :
        sumtotal = tradeTotal + row['P / L']
        nt.at[i, 'Sum'] = sumtotal
        tradeTotal = 0
nt.head()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
4,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
5,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,,
6,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
7,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
8,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,,


In [40]:
import datetime

In [41]:
#Use filter instead of iteration
# def addTradeDuration 
tradeTotal = 0.0
for i, row in nt.iterrows():
    if row['Balance'] == 0 :
        timeEnd = row['Time']
        timeStart = row['Start']
        end=timeEnd.split(":")
        start=timeStart.split(":")
        diff = datetime.datetime(1,1,1,int(end[0]), int(end[1]), int(end[2])) - datetime.datetime(1,1,1,int(start[0]), int(start[1]), int(start[2]))
        nt.at[i,'Duration'] = diff
nt.head()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
4,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
5,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,0:00:29,
6,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
7,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
8,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,0:03:24,


In [42]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
4,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
5,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,0:00:29,
6,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
7,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
8,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,0:03:24,
0,Trade 3,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
1,Trade 3,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
2,Trade 3,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
3,Trade 3,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,115.06,14:15:05,
9,Trade 4,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


In [43]:
# def addTradeName
for i, row in nt.iterrows():
    longShort = " Long"
    if row['Balance'] == 0 :
        if row['Side'] == 'B' :
            longShort = " Short"
        nt.at[i, 'Name'] = row['Symb'] + longShort
nt.head()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
4,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
5,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,0:00:29,MU Short
6,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
7,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
8,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,0:03:24,MU Long


In [44]:
nt=addRows(nt,1)
nt.tail(10)

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
13,Trade 5,10:00:10,10:00:13,MU,B,46.92,200,763,TRIB5621,0,,,
14,Trade 5,10:00:10,11:54:38,MU,S,47.15,-763,0,TRIB5621,164.23,164.23,1:54:28,MU Long
15,Trade 6,13:54:52,13:54:52,WMT,B,98.2,425,425,TRIB5621,0,,,
16,Trade 6,13:54:52,13:56:01,WMT,S,98.01,-425,0,TRIB5621,-80.75,-80.75,0:01:09,WMT Long
17,Trade 7,14:15:08,14:15:08,ROKU,B,55.58,265,265,TRIB5621,0,,,
18,Trade 7,14:15:08,14:19:13,ROKU,S,55.46,-265,0,TRIB5621,-31.8,-31.8,0:04:05,ROKU Long
19,Trade 8,15:11:28,15:11:28,MU,SS,47.18,-1050,-1050,TRIB5621,0,,,
20,Trade 8,15:11:28,15:44:39,MU,B,47.21,300,-750,TRIB5621,-9,,,
21,Trade 8,15:11:28,23:59:59,MU,HOLD-,0,-750,0,ZeroSubstance,0,-9,8:48:31,MU Long
22,,,,,,,,,,,,,


### This will blow up from wrong types if there is some kind of anomaly. The blank rows in Sum col are str and the filled rows are float. Don't fix it until it blows up. (or you need change this 'now' to production code for some reason) That way we have some case to fix.
### Also need to fix below if the lastCol for Balance > 0 instead of what's there

In [45]:
# nt['P / L'].sum()
nt['P / L'].dtype

dtype('O')

In [46]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
1,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,0:00:29,MU Short
2,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
3,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
4,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,0:03:24,MU Long
5,Trade 3,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
6,Trade 3,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
7,Trade 3,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
8,Trade 3,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,115.06,14:15:05,IZEA Long
9,Trade 4,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


In [47]:
# def addSummaryPL
count=0
tot=0.0
tot2 = 0.0
for i, row in nt.iterrows():
    count=count+1
    if count < len(nt) :
        tot=tot+row['P / L']
        if row['Balance'] == 0 :
#             assert(type(row['Sum']) == type(1.1))
            tot2 = tot2 + row['Sum']
        if count == len(nt) -1 :
            lastCol = row['P / L']
#             assert(type(lastCol) == type(1.1))
            print("Last col?", row['P / L'])
#         print(count, ": ", tot, tot2)
    else :
        nt.at[i, 'P / L'] = tot
        nt.at[i, 'Sum'] = tot2
#         print("Grand total: ", tot, tot2)

# if(tot != tot2) or lastCol > 0:

if lastCol > 0:

    print('''
    I believe you either closed an overnight trade or you 
    began or changed a previous trade.  I tried to fix it 
    earlier,, but evidently I didn't account for something :)
    ''')
nt



Last col? 0


,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
1,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,0:00:29,MU Short
2,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
3,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
4,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,0:03:24,MU Long
5,Trade 3,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
6,Trade 3,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
7,Trade 3,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
8,Trade 3,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,115.06,14:15:05,IZEA Long
9,Trade 4,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


In [48]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
1,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,0:00:29,MU Short
2,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
3,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
4,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,0:03:24,MU Long
5,Trade 3,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
6,Trade 3,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
7,Trade 3,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
8,Trade 3,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,115.06,14:15:05,IZEA Long
9,Trade 4,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


In [49]:
# ldf = list()
# count=0
# tot=0.0
# tot2 = 0.0
# tempdf = pd.DataFrame()
# for i, row in nt.iterrows():
#     count=count+1
#     if count < len(nt) :
#         tot=tot+row['P / L']
#         if row['Balance'] == 0 :
#             assert(type(row['Sum']) == type(1.1))
#             tot2 = tot2 + row['Sum']
#         if count == len(nt) -1 :
#             lastCol = row['P / L']
#             assert(type(lastCol) == type(1.1))
#             print("Last col?", row['P / L'])
# #         print(count, ": ", tot, tot2)
#     else :
#         nt.at[i, 'P / L'] = tot
#         nt.at[i, 'Sum'] = tot2
# #         print("Grand total: ", tot, tot2)

# # if(tot != tot2) or lastCol > 0:

# if lastCol > 0:

#     print('''
#     I believe you either closed an overnight trade or you 
#     began or changed a previous trade.  Your on your own to 
#     fix that. I am going to blithly continue as if nothing 
#     happened :)
#     ''')
# nt.tail()



In [50]:
# ldf = list()
# count = 1
# while True :
#     tradeStr = "Trade " + str(count)
#     count = count + 1
#     tdf = nt[nt.Tindex == tradeStr]
#     if len(tdf) > 0 :
#         ldf.append(tdf)
#     else :
#         break
# assert (len(ldf) == numTrades)
# len(ldf)


In [51]:
# numTransactions = len(ldf[0])
# print (numTransactions)
# for i in range(numTransactions, numTransactions + )
# # ldf[0] = addRows(ldf[0], 20)
# ldf[0]

In [52]:
# for tdf in ldf :
#     nt = nt.append(tdf, ignore_index = True)
#     tdf = addRows(tdf, 20)
#     print(count, ": ", len(nt))

In [53]:
ldf = list()
count = 1
while True :
    tradeStr = "Trade " + str(count)
    count = count + 1
    tdf = nt[nt.Tindex == tradeStr]
    if len(tdf) > 0 :
        ldf.append(tdf)
    else :
        break
len(ldf)


8

In [54]:
nt = addRows(nt, 2)
nt.tail()

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
20,Trade 8,15:11:28,15:44:39,MU,B,47.21,300,-750,TRIB5621,-9,,,
21,Trade 8,15:11:28,23:59:59,MU,HOLD-,0,-750,0,ZeroSubstance,0,-9,8:48:31,MU Long
22,,,,,,,,,,-223.44,-223.44,,
23,,,,,,,,,,,,,
24,,,,,,,,,,,,,


In [55]:
for tdf in ldf :
    nt = nt.append(tdf, ignore_index = True)
    nt = addRows(nt, 20)
    print(count, ": ", len(nt))

10 :  47
10 :  70
10 :  94
10 :  116
10 :  140
10 :  162
10 :  184
10 :  207


In [56]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
1,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,0:00:29,MU Short
2,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
3,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
4,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,0:03:24,MU Long
5,Trade 3,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
6,Trade 3,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
7,Trade 3,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
8,Trade 3,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,115.06,14:15:05,IZEA Long
9,Trade 4,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


In [57]:
name=datetime.date.today().strftime("Trades_%A_%m%d.csv")
nt.to_csv(name)

In [58]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
1,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,0:00:29,MU Short
2,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
3,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
4,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,0:03:24,MU Long
5,Trade 3,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
6,Trade 3,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
7,Trade 3,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
8,Trade 3,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,115.06,14:15:05,IZEA Long
9,Trade 4,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,


In [59]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,Trade 1,09:35:33,09:35:33,MU,SS,45.77,-1000,-1000,TRIB5621,0,,,
1,Trade 1,09:35:33,09:36:02,MU,B,46,1000,0,TRIB5621,-230,-230,0:00:29,MU Short
2,Trade 2,09:37:48,09:37:48,MU,B,46.3,1000,1000,TRIB5621,0,,,
3,Trade 2,09:37:48,09:38:46,MU,S,46.44,-250,750,TRIB5621,35,,,
4,Trade 2,09:37:48,09:41:12,MU,S,46.32,-750,0,TRIB5621,15,50,0:03:24,MU Long
5,Trade 3,09:44:54,09:44:54,IZEA,B,1.63,6016,6016,TRIB5621,0,,,
6,Trade 3,09:44:54,09:45:21,IZEA,B,1.59,1830,7846,TRIB5621,0,,,
7,Trade 3,09:44:54,09:47:48,IZEA,S,1.65,-3923,3923,TRIB5621,115.06,,,
8,Trade 3,09:44:54,23:59:59,IZEA,HOLD+,0,3923,0,ZeroSubstance,0,115.06,14:15:05,IZEA Long
9,Trade 4,09:53:58,09:53:58,MU,SS,46.46,-479,-479,TRIB5621,0,,,
